In [15]:
import pandas as pd
import sys
import os
import configparser
import requests
import json

In [16]:
repo = os.path.abspath(os.pardir)

In [17]:
sys.path.append(repo)

In [18]:
config = configparser.ConfigParser()

In [19]:
config.read(os.path.join(repo,'config.ini'))

['c:\\Users\\kiera\\OneDrive\\Documents\\School\\Summer 24\\ECE 612\\ECE612_HW3\\config.ini']

In [20]:
cx = config.get('Custom Search','cx')
key = config.get('Custom Search','API_key')

In [21]:
req = requests.get(f"https://www.googleapis.com/customsearch/v1?key={key}&cx={cx}&imgType=stock&q=bird in birdfeeder picture",)
decoded_req = json.loads(req.content.decode())

In [90]:
decoded_req.keys()

dict_keys(['kind', 'url', 'queries', 'context', 'searchInformation', 'spelling', 'items'])

{'kind': 'customsearch#result',
 'title': '30,000+ Bird Feeder Pictures | Download Free Images on Unsplash',
 'htmlTitle': '30,000+ <b>Bird Feeder Pictures</b> | Download Free Images on Unsplash',
 'link': 'https://unsplash.com/s/photos/bird-feeder',
 'displayLink': 'unsplash.com',
 'snippet': 'Download the perfect bird feeder pictures. Find over 100+ of the best free bird feeder images. Free for commercial use ✓ No attribution required\xa0...',
 'htmlSnippet': 'Download the perfect <b>bird feeder pictures</b>. Find over 100+ of the best free <b>bird feeder images</b>. Free for commercial use ✓ No attribution required&nbsp;...',
 'formattedUrl': 'https://unsplash.com/s/photos/bird-feeder',
 'htmlFormattedUrl': 'https://unsplash.com/s/photos/<b>bird</b>-feeder',
 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSW2qfDhRyc8CchKzxLRNoHcfOAxfGdi2lSOZBjzpaaIdmWTmRgtM8QDaE&s',
    'width': '310',
    'height': '163'}],
  'imageobject': [{'contentur

'https://media.gettyimages.com/id/155599045/photo/busy-bird-feeder.jpg?s=612x612&w=gi&k=20&c=A9blS6HKNt0MlVB7Af1TNTAXWE4zSv3FZ9n2Phco0cE='

In [63]:
import shutil

def save_image_from_url(url, file_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(file_path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        print(f"Image saved to {file_path}")
    else:
        print(f"Failed to retrieve image from {url}")
    del response

In [65]:
save_image_from_url(decoded_req['items'][4]['pagemap']['imageobject'][0]['thumbnailurl'],os.path.join(repo,'data/raw/birder_training_images/sample_image_1.jpg'))

Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_1.jpg


In [101]:
for ind in range(len(decoded_req['items'][9]['pagemap']['imageobject'])):
    save_image_from_url(decoded_req['items'][9]['pagemap']['imageobject'][ind]['thumbnailurl'],os.path.join(repo,f'data/raw/birder_training_images/sample_image_{ind+50}.jpg'))

Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_50.jpg
Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_51.jpg
Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_52.jpg
Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_53.jpg
Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_54.jpg
Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_55.jpg
Image saved to c:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data/raw/birder_training_images/sample_image_56.jpg
Image saved to c:\Users\kiera\OneDrive\Do

In [22]:
data_path=os.path.join(repo,'data','raw','birder_training_images')

In [23]:
from matplotlib.image import imread
import tensorflow as tf

In [25]:
import pandas as pd
labels = pd.read_csv(r'C:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data\raw\birder_training_images\labels.csv')
labels.set_index('img_index',inplace=True)

In [26]:
labels

,label
img_index,
0,1
1,1
2,0
3,1
4,1
...,...
65,1
66,0
67,1


In [37]:
images=[]
label=[]
import os
for index in labels.index:
    images.append(imread(os.path.join(data_path,f'sample_image_{index}.jpg')))
    label.append(labels.iloc[index,0])

In [38]:
labels.iloc[0,0]

1

In [41]:
feature_description={'image':tf.io.FixedLenFeature([],tf.string),
'birdType':tf.io.FixedLenFeature([],tf.int64)}
from tensorflow.train import Feature, Features, Example, BytesList, Int64List
with tf.io.TFRecordWriter(r'C:\Users\kiera\OneDrive\Documents\School\Summer 24\ECE 612\ECE612_HW3\data\processed\birder_training_images\birder_dataset.tfrecords') as writer:
    for indiv_image,indiv_label in zip(images,label):
        encoded_image = tf.io.encode_jpeg(indiv_image).numpy()
        feature_dict = {
            "image": Feature(bytes_list=BytesList(value=[encoded_image])),
            "feederandbird": Feature(int64_list=Int64List(value=[indiv_label]))
        }
        example = Example(features=Features(feature=feature_dict))
        writer.write(example.SerializeToString())

In [32]:
label

[label    1
 Name: 0, dtype: int64,
 label    1
 Name: 1, dtype: int64,
 label    0
 Name: 2, dtype: int64,
 label    1
 Name: 3, dtype: int64,
 label    1
 Name: 4, dtype: int64,
 label    1
 Name: 5, dtype: int64,
 label    1
 Name: 6, dtype: int64,
 label    1
 Name: 7, dtype: int64,
 label    1
 Name: 8, dtype: int64,
 label    1
 Name: 9, dtype: int64,
 label    1
 Name: 10, dtype: int64,
 label    1
 Name: 11, dtype: int64,
 label    1
 Name: 12, dtype: int64,
 label    1
 Name: 13, dtype: int64,
 label    1
 Name: 14, dtype: int64,
 label    1
 Name: 15, dtype: int64,
 label    1
 Name: 16, dtype: int64,
 label    1
 Name: 17, dtype: int64,
 label    0
 Name: 18, dtype: int64,
 label    0
 Name: 19, dtype: int64,
 label    0
 Name: 20, dtype: int64,
 label    0
 Name: 21, dtype: int64,
 label    1
 Name: 22, dtype: int64,
 label    1
 Name: 23, dtype: int64,
 label    1
 Name: 24, dtype: int64,
 label    1
 Name: 25, dtype: int64,
 label    1
 Name: 26, dtype: int64,
 label    1